In [1]:
# all init
import sympy as sp
from sympy.vector import CoordSys3D
from sympy.matrices import Matrix
import numpy as np
from IPython.display import display
import time
sp.init_printing(use_latex=True)

In [2]:
epsoct, eta = sp.symbols(r'\epsilon_{oct} \eta', positive=True)
N = CoordSys3D('N')
ex, ey, ez, jx, jy, jz = sp.symbols(r'e_x e_y e_z j_x j_y j_z', real=True)
e2x, e2y, e2z, j2x, j2y, j2z = sp.symbols(r'e_{2x} e_{2y} e_{2z} j_{2x} j_{2y} j_{2z}', real=True)

evec = ex * N.i + ey * N.j + ez * N.k
jvec = jx * N.i + jy * N.j + jz * N.k
e2vec = e2x * N.i + e2y * N.j + e2z * N.k
j2vec = j2x * N.i + j2y * N.j + j2z * N.k

esq, e2sq, jsq, j2sq = sp.symbols('e2 e2_2 j2 j2_2', positive=True, real=True)
nvec = jvec / sp.sqrt(jsq)
n2vec = j2vec / sp.sqrt(j2sq)
uvec = evec / sp.sqrt(esq)
u2vec = e2vec / sp.sqrt(e2sq)
jsubs = {
    esq: ex**2 + ey**2 + ez**2,
    e2sq: e2x**2 + e2y**2 + e2z**2,
    jsq: jx**2 + jy**2 + jz**2,
    j2vec: j2x**2 + j2y**2 + j2z**2,
}

In [3]:
# mu1 = phi0 = L1 = 1, absorbed into t_K + epsoct

phi_quad = (1 / (8 * j2sq**(sp.Rational(3, 2))) * (
    1
    - 6 * esq
    - 3 * (jvec.dot(n2vec))**2
    + 15 * (evec.dot(n2vec))**2
)).subs(jsubs)
phi_oct = (15 * epsoct / (64 * j2sq**(sp.Rational(3, 2)))* (
    evec.dot(u2vec) * (
        8 * esq
        - 1
        - 35 * (evec.dot(n2vec))**2
        + 5 * (jvec.dot(n2vec))**2
    )
    + 10 * evec.dot(n2vec) * jvec.dot(u2vec) * jvec.dot(n2vec)
)).subs(jsubs)
phi = phi_quad + phi_oct
def dvec(phi, vec):
    return (
        sp.diff(phi, vec.coeff(N.i)) * N.i
        + sp.diff(phi, vec.coeff(N.j)) * N.j
        + sp.diff(phi, vec.coeff(N.k)) * N.k
    )
djvecdt = -(jvec.cross(dvec(phi, jvec)) + evec.cross(dvec(phi, evec)))
devecdt = -(jvec.cross(dvec(phi, evec)) + evec.cross(dvec(phi, jvec)))
dj2vecdt = -eta * (j2vec.cross(dvec(phi, j2vec)) + e2vec.cross(dvec(phi, e2vec)))
de2vecdt = -eta * (j2vec.cross(dvec(phi, e2vec)) + e2vec.cross(dvec(phi, j2vec)))

In [4]:
djvecdt_exact = (sp.Rational(3, 4) / j2sq**(sp.Rational(3, 2)) * (
    jvec.dot(n2vec) * jvec.cross(n2vec)
    - 5 * evec.dot(n2vec) * evec.cross(n2vec)
)) + (epsoct * 75 / (64 * j2sq**(sp.Rational(3, 2))) * (
    n2vec.cross(
        2 * (evec.dot(u2vec) * jvec.dot(n2vec)
            + evec.dot(n2vec) * jvec.dot(u2vec)) * jvec
        + 2 * (jvec.dot(u2vec) * jvec.dot(n2vec)
            - 7 * evec.dot(u2vec) * evec.dot(n2vec)) * evec
    )
    + u2vec.cross(
        2 * (evec.dot(n2vec) * jvec.dot(n2vec)) * jvec
        + (8 * esq / 5 - sp.Rational(1, 5) - 7 * (evec.dot(n2vec))**2 + (jvec.dot(n2vec))**2) * evec
    )
)).subs(jsubs)

print(sp.simplify((djvecdt_exact - djvecdt).coeff(N.i)))
print(sp.simplify((djvecdt_exact - djvecdt).coeff(N.j)))
print(sp.simplify((djvecdt_exact - djvecdt).coeff(N.k)))

0
0
0


In [5]:
devecdt_exact = (sp.Rational(3, 4) / j2sq**(sp.Rational(3, 2)) * (
    jvec.dot(n2vec) * evec.cross(n2vec)
    - 5 * evec.dot(n2vec) * jvec.cross(n2vec)
    + 2 * jvec.cross(evec)
)) + (epsoct * 75 / (64 * j2sq**(sp.Rational(3, 2))) * (
    u2vec.cross(
        2 * (evec.dot(n2vec) * jvec.dot(n2vec)) * evec
        + (8 * esq / 5
           - sp.Rational(1, 5)
           - 7 * (evec.dot(n2vec))**2
           + (jvec.dot(n2vec))**2
          ) * jvec
    )
    + n2vec.cross(
        2 * (evec.dot(u2vec) * jvec.dot(n2vec)
            + evec.dot(n2vec) * jvec.dot(u2vec)) * evec
        + 2 * (jvec.dot(n2vec) * jvec.dot(u2vec)
            - 7 * evec.dot(n2vec) * evec.dot(u2vec)) * jvec
    )
    - sp.Rational(16, 5) * evec.dot(u2vec) * jvec.cross(evec)
)).subs(jsubs)
print(sp.simplify((devecdt_exact - devecdt).coeff(N.i)))

0


### Try generating some C code for ourselves for the orbital elements, to verify against Bin's code

In [6]:
variable_namer = sp.numbered_symbols('x_')
equations = [
    djvecdt.coeff(N.i),
    djvecdt.coeff(N.j),
    djvecdt.coeff(N.k),
    devecdt.coeff(N.i),
    devecdt.coeff(N.j),
    devecdt.coeff(N.k),
    dj2vecdt.coeff(N.i),
    dj2vecdt.coeff(N.j),
    dj2vecdt.coeff(N.k),
    de2vecdt.coeff(N.i),
    de2vecdt.coeff(N.j),
    de2vecdt.coeff(N.k),
]
replacements, reduced = sp.cse(equations, symbols=variable_namer)

```
print('return [')
for i, r in enumerate(equations):
    print('\t' + sp.ccode(sp.expand(r)).replace('\\', '').replace('{', '').replace('}', '') + ',')
print(']')
```

In [7]:
for key, val in replacements:
    print('cdef double', key, '=', sp.ccode(val).replace('\\', '').replace('{', '').replace('}', ''))

print()
print('return [')
for i, r in enumerate(reduced):
    print('\t' + sp.ccode(r).replace('\\', '').replace('{', '').replace('}', '') + ',')
print(']')

cdef double x_0 = pow(j2_2, -1.0/2.0)
cdef double x_1 = j_2x*x_0
cdef double x_2 = j_2y*x_0
cdef double x_3 = j_2z*x_0
cdef double x_4 = j_x*x_1 + j_y*x_2 + j_z*x_3
cdef double x_5 = (3.0/4.0)*x_4/pow(j2_2, 2)
cdef double x_6 = j_2z*x_5
cdef double x_7 = pow(e_2x, 2)
cdef double x_8 = pow(e_2y, 2)
cdef double x_9 = pow(e_2z, 2)
cdef double x_10 = x_7 + x_8 + x_9
cdef double x_11 = pow(x_10, -1.0/2.0)
cdef double x_12 = e_x*x_1
cdef double x_13 = e_y*x_2
cdef double x_14 = e_z*x_3
cdef double x_15 = 10*x_12 + 10*x_13 + 10*x_14
cdef double x_16 = x_15*x_4
cdef double x_17 = x_11*x_16
cdef double x_18 = e_x*x_11
cdef double x_19 = e_y*x_11
cdef double x_20 = e_z*x_11
cdef double x_21 = e_2x*x_18 + e_2y*x_19 + e_2z*x_20
cdef double x_22 = 10*x_4
cdef double x_23 = x_21*x_22
cdef double x_24 = j_x*x_11
cdef double x_25 = j_y*x_11
cdef double x_26 = j_z*x_11
cdef double x_27 = e_2x*x_24 + e_2y*x_25 + e_2z*x_26
cdef double x_28 = x_15*x_27
cdef double x_29 = pow(j2_2, -3.0/2.0)
cdef double x_

Compared to Bin's code: de2_dt seems to be wrong, the rest seem to differ at the ~0.1% level. Note: timestepper advances too aggressively with t / t_lk units, unit jvecs, requires much higher tol, prefer Bin's code

# Now, for the orbital element transformation

In [8]:
t = sp.symbols('t')

et = sp.Function(r'e')(t)
It = sp.Function(r'I')(t)
Wt = sp.Function(r'\Omega')(t)
wt = sp.Function(r'\omega')(t)
e2t = sp.Function(r'e_2')(t)
I2t = sp.Function(r'I_2')(t)
W2t = sp.Function(r'\Omega_2')(t)
w2t = sp.Function(r'\omega_2')(t)

jvecx_orbs = sp.sqrt(1 - et**2) * sp.sin(It) * sp.sin(Wt)
jvecy_orbs = -sp.sqrt(1 - et**2) * sp.sin(It) * sp.cos(Wt)
jvecz_orbs = sp.sqrt(1 - et**2) * sp.cos(It)
j2vecx_orbs = sp.sqrt(1 - e2t**2) * sp.sin(I2t) * sp.sin(W2t)
j2vecy_orbs = -sp.sqrt(1 - e2t**2) * sp.sin(I2t) * sp.cos(W2t)
j2vecz_orbs = sp.sqrt(1 - e2t**2) * sp.cos(I2t)
evecx_orbs = et * (sp.cos(wt) * sp.cos(Wt) - sp.sin(wt) * sp.cos(It) * sp.sin(Wt))
evecy_orbs = et * (sp.cos(wt) * sp.sin(Wt) + sp.sin(wt) * sp.cos(It) * sp.cos(Wt))
evecz_orbs = et * sp.sin(wt) * sp.sin(It)
e2vecx_orbs = e2t * (sp.cos(w2t) * sp.cos(W2t) - sp.sin(w2t) * sp.cos(I2t) * sp.sin(W2t))
e2vecy_orbs = e2t * (sp.cos(w2t) * sp.sin(W2t) + sp.sin(w2t) * sp.cos(I2t) * sp.cos(W2t))
e2vecz_orbs = e2t * sp.sin(w2t) * sp.sin(I2t)

In [9]:
e, I, W, w, e2, I2, W2, w2 = sp.symbols(r'e I \Omega \omega e_2 I_2 \Omega_2 \omega_2', real=True)
func_subsdict = {
    et: e,
    It: I,
    Wt: W,
    wt: w,
    e2t: e2,
    I2t: I2,
    W2t: W2,
    w2t: w2,
}

system = [
    sp.diff(jvecy_orbs, t),
    sp.diff(jvecz_orbs, t),
    sp.diff(evecy_orbs, t),
    sp.diff(evecz_orbs, t),
    sp.diff(j2vecy_orbs, t),
    sp.diff(j2vecz_orbs, t),
    sp.diff(e2vecy_orbs, t),
    sp.diff(e2vecz_orbs, t),
]

# build coord transformation matrix
M = Matrix(np.zeros((8, 8)))
sub_keys = list(func_subsdict.keys())
for col, key1 in enumerate(sub_keys):
    subs_temp = {}
    for idx, key2 in enumerate(sub_keys):
        if idx == col:
            subs_temp[sp.Derivative(key2, t)] = 1
        else:
            subs_temp[sp.Derivative(key2, t)] = 0
    for row, eq in enumerate(system):
        M[row, col] = eq.subs(subs_temp).subs(func_subsdict)

f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12 = sp.symbols('f1 f2 f3 f4 f5 f6 f7 f8 f9 f10 f11 f12', real=True)
v = Matrix([f2, f3, f5, f6, f8, f9, f11, f12])
ret = M.LUsolve(v)

# Theoretically Done

Let's check whether it works. The 8 rows correspond to the 8 derivatives in func_subsdict, while the f1-8 correspond to the to-be-substituted forms from the vector equations

In [10]:
# first, replacements the fs
orbsubs = {
    jx: jvecx_orbs.subs(func_subsdict),
    jy: jvecy_orbs.subs(func_subsdict),
    jz: jvecz_orbs.subs(func_subsdict),
    ex: evecx_orbs.subs(func_subsdict),
    ey: evecy_orbs.subs(func_subsdict),
    ez: evecz_orbs.subs(func_subsdict),
    j2x: j2vecx_orbs.subs(func_subsdict),
    j2y: j2vecy_orbs.subs(func_subsdict),
    j2z: j2vecz_orbs.subs(func_subsdict),
    e2x: e2vecx_orbs.subs(func_subsdict),
    e2y: e2vecy_orbs.subs(func_subsdict),
    e2z: e2vecz_orbs.subs(func_subsdict),

}
fsubs = {
    f2: equations[1],
    f3: equations[2],
    f5: equations[4],
    f6: equations[5],
    f8: equations[7],
    f9: equations[8],
    f11: equations[10],
    f12: equations[11],
}

f_eqs = [val.subs(orbsubs) for val in fsubs.values()]
variable_namer3 = sp.numbered_symbols('z_')
replacements3, reduced3 = sp.cse(f_eqs, symbols=variable_namer3)

for key, val in replacements3:
    print('cdef double', key, '=', sp.ccode(val).replace('\\', '').replace('{', '').replace('}', ''))
    
for key, replacement in zip(fsubs.keys(), reduced3):
    print('cdef double', key, '=', sp.ccode(replacement).replace('\\', '').replace('{', '').replace('}', ''))

# next, replacements the solutions
variable_namer2 = sp.numbered_symbols('y_')
replacements2, reduced2 = sp.cse(list(ret), symbols=variable_namer2)

for key, val in replacements2:
    print('cdef double', key, '=', sp.ccode(val).replace('\\', '').replace('{', '').replace('}', ''))

# finally, print reduced solutions

print("#", ' '.join([str(func_subsdict[k]) for k in sub_keys]))
print('return [')
for i, r in enumerate(reduced2):
    print('\t' + sp.ccode(r).replace('\\', '').replace('{', '').replace('}', '') + ',')
print(']')

cdef double z_0 = cos(I_2)
cdef double z_1 = pow(e_2, 2)
cdef double z_2 = sqrt(1 - z_1)
cdef double z_3 = z_0*z_2
cdef double z_4 = cos(I)
cdef double z_5 = pow(e, 2)
cdef double z_6 = sqrt(1 - z_5)
cdef double z_7 = z_4*z_6
cdef double z_8 = pow(j2_2, -1.0/2.0)
cdef double z_9 = z_3*z_8
cdef double z_10 = sin(Omega)
cdef double z_11 = sin(I)
cdef double z_12 = z_11*z_6
cdef double z_13 = z_10*z_12
cdef double z_14 = sin(Omega_2)
cdef double z_15 = sin(I_2)
cdef double z_16 = z_15*z_2
cdef double z_17 = z_14*z_16
cdef double z_18 = z_17*z_8
cdef double z_19 = cos(Omega)
cdef double z_20 = z_12*z_19
cdef double z_21 = cos(Omega_2)
cdef double z_22 = z_16*z_21
cdef double z_23 = z_22*z_8
cdef double z_24 = z_13*z_18 + z_20*z_23 + z_7*z_9
cdef double z_25 = (3.0/4.0)*z_24/pow(j2_2, 2)
cdef double z_26 = sin(omega_2)
cdef double z_27 = z_15*z_26
cdef double z_28 = e_2*z_27
cdef double z_29 = z_1*pow(z_15, 2)*pow(z_26, 2)
cdef double z_30 = cos(omega_2)
cdef double z_31 = z_0*z_26
cdef dou

# Conclusion
To absolutely nobody's surprise, this is a disaster. Instead, let's try getting the equations by transforming the equations in Naoz's 2013 paper.